In [3]:
import numpy as np
import xml.etree.ElementTree as ET
from IPython.display import display
import matplotlib.pyplot as plt
import sklearn
import os
from gensim import corpora


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import spacy

c:\users\waii\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\users\waii\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\waii\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
c:\users\waii\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\waii\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, ma

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\waii\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
stemmer = PorterStemmer()
lemma = nltk.stem.WordNetLemmatizer()
def lemmatize_stemming(text):
#     return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    return lemma.lemmatize(text)

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lemmatize_stemming(token))
    return result

In [6]:
dir_leng = len(os.listdir('./category'))
category_dict = {}
i = 0
for file in os.listdir('./category'):
    i += 1
    print(str((i/dir_leng)*100),end="\r")
#     print(file,len(file))
    pmid_set = set()
    read_file = open('./category/'+file).read().splitlines()
    for r in read_file:
        pmid_set.add(r.strip())
    category_dict[file[:len(file)-13]] = list(pmid_set)
  

In [32]:
pos_abstract = []
i = 0
dir_leng = len(os.listdir('./src/train/find_pmid_with_abstract'))
print(dir_leng)
# cat = set()

for file in os.listdir('./src/train/find_pmid_with_abstract'):
    i += 1
    print(str((i/dir_leng)*100),end="\r")
    et = ET.parse('./src/train/find_pmid_with_abstract/'+file)
    root = et.getroot()
    article = root.find('MedlineCitation').find('Article').findall('Abstract')
    pmid = root.find('MedlineCitation').findall('PMID')
# #     print(pmid)
#     if len(pmid) > 1:
#         print(pmid)
    for category,p_list in category_dict.items():
#         print(category)
        for p in pmid:
            if p.text in p_list:
#                 print(p.text)
#                 cat.add(category)
                word = {}
                for a in article:
                    atext = a.find('AbstractText').text
#                     print(atext)
                    if type(atext) == str:
#                         print(atext)
                        pos_abstract.append([atext,category])

22121


In [8]:
token_without_category = []
i = 0

for pabs in pos_abstract:   
    i += 1
    print(str((i/len(pos_abstract))*100),end="\r")
    token_without_category.append(preprocess(pabs[0]))

In [7]:
freq = {}
remove_list = ['ii','iii','use','http','available','data','based','tool','method','information','study','new','high','use','data','user','make','according','result']


for r in remove_list:
    for t in token_without_category:
        if r in t:
            t.remove(r)

for t in range(len(token_without_category)):
    for word in range(len(token_without_category[t])):
        if token_without_category[t][word] not in freq:
            freq[token_without_category[t][word]] = 0
        freq[token_without_category[t][word]] += 1

In [8]:
sorted(freq, key=freq.get, reverse=True)

['protein',
 'gene',
 'sequence',
 'analysis',
 'database',
 'genome',
 'structure',
 'model',
 'data',
 'set',
 'rna',
 'method',
 'prediction',
 'approach',
 'sequencing',
 'algorithm',
 'expression',
 'network',
 'web',
 'cell',
 'site',
 'interaction',
 'developed',
 'human',
 'large',
 'function',
 'number',
 'different',
 'disease',
 'present',
 'software',
 'annotation',
 'biological',
 'alignment',
 'target',
 'functional',
 'dna',
 'provides',
 'specific',
 'feature',
 'multiple',
 'binding',
 'genomic',
 'complex',
 'search',
 'tool',
 'important',
 'based',
 'single',
 'specie',
 'region',
 'computational',
 'process',
 'including',
 'type',
 'sample',
 'resource',
 'novel',
 'identification',
 'genetic',
 'application',
 'pathway',
 'structural',
 'cancer',
 'read',
 'time',
 'interface',
 'molecular',
 'variant',
 'program',
 'server',
 'provide',
 'level',
 'drug',
 'experimental',
 'known',
 'identify',
 'information',
 'research',
 'www',
 'source',
 'peptide',
 'accura

In [9]:
dictionary = corpora.Dictionary(token_without_category)

In [10]:
corpus = [dictionary.doc2bow(t) for t in token_without_category]

In [11]:
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('lda.gensim')

In [12]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)

c:\users\waii\appdata\local\programs\python\python36\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [13]:
pyLDAvis.display(lda_display)

In [14]:
topics = ldamodel.print_topics(num_words=4)

for topic in topics:
    print(topic)

(0, '0.037*"binding" + 0.030*"ligand" + 0.024*"molecule" + 0.021*"site"')
(1, '0.035*"m" + 0.034*"peptide" + 0.033*"mass" + 0.025*"spectrum"')
(2, '0.033*"cell" + 0.027*"expression" + 0.026*"gene" + 0.016*"cancer"')
(3, '0.055*"database" + 0.018*"annotation" + 0.016*"resource" + 0.014*"gene"')
(4, '0.056*"cancer" + 0.055*"mutation" + 0.035*"tumor" + 0.031*"cell"')
(5, '0.034*"variant" + 0.025*"genetic" + 0.022*"disease" + 0.019*"association"')
(6, '0.037*"genome" + 0.033*"sequencing" + 0.030*"sequence" + 0.021*"read"')
(7, '0.145*"protein" + 0.039*"interaction" + 0.015*"function" + 0.015*"site"')
(8, '0.022*"analysis" + 0.019*"data" + 0.019*"web" + 0.016*"tool"')
(9, '0.064*"gene" + 0.025*"specie" + 0.024*"genome" + 0.023*"tree"')
(10, '0.039*"gene" + 0.021*"site" + 0.019*"genome" + 0.017*"binding"')
(11, '0.044*"seq" + 0.035*"structure" + 0.016*"protein" + 0.011*"energy"')
(12, '0.039*"prediction" + 0.026*"method" + 0.016*"feature" + 0.015*"accuracy"')
(13, '0.053*"sequence" + 0.049*"

In [15]:
stemmer = PorterStemmer()
lemma = nltk.stem.WordNetLemmatizer()
def lemmatize_stemming(text):
#     return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    return lemma.lemmatize(text)

def preprocess(text):
    result = []
    remove_list = ['ii','iii','use','http','available','data','based','tool','method','information','study','new','high','use','data','user','make','according','result','incorrectly','usually','miss']
#     remove_list = [':']
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and token not in remove_list:
            result.append(lemmatize_stemming(token))
    return result

In [16]:
pos_title = []
i = 0
dir_leng = len(os.listdir('./src/train/find_pmid_with_abstract'))
print(dir_leng)
cat = set()

for file in os.listdir('./src/train/find_pmid_with_abstract'):
    i += 1
    print(str((i/dir_leng)*100),end="\r")
    et = ET.parse('./src/train/find_pmid_with_abstract/'+file)
    root = et.getroot()
    title = root.find('MedlineCitation').find('Article').findall('ArticleTitle')
    pmid = root.find('MedlineCitation').findall('PMID')
#     print(pmid)
#     if len(pmid) > 1:
#         print(pmid)
    for category,p_list in category_dict.items():
#         print(category)
        for p in pmid:
            if p.text in p_list:
#                 print(p.text)
                cat.add(category)
                word = {}
                for a in title:
                    atext = a.text
#                     print(atext)
                    if type(atext) == str:
#                         print(atext)
                        pos_title.append([atext,category])

22121


In [17]:
token_with_category = []
i = 0

for pabs in pos_title:   
    i += 1
    print(str((i/len(pos_title))*100),end="\r")
    token_with_category.append([preprocess(pabs[0]),pabs[1]])

In [33]:
atoken_with_category = []
i = 0

for pabs in pos_abstract:   
    i += 1
    print(str((i/len(pos_abstract))*100),end="\r")
    atoken_with_category.append([preprocess(pabs[0]),pabs[1]])

96.65409143986346555522.68031977005299628.7388843977364634.8693074642953440.8560136531033846.1465912153058584.3258780203000194.41300637743645

In [18]:
category_file = open('category_pmid_count_minimal.txt','r').readlines()

In [19]:
top20_category = {}
for cat in category_file:
    split_str = cat.split('\t')
    top20_category[split_str[0]] = int(split_str[1].strip('\n'))
top20_category = sorted(top20_category.items(),key=lambda kv: kv[1],reverse=True)[0:20]
top20_category_first = [t[0] for t in top20_category]

In [20]:
vocab = []
i = 0
for token in token_with_category:
    i += 1
    print(str((i/len(token_with_category))*100),end='\r')
    if str(token[1]+'-category') in top20_category_first:
        for t in token[0]:
            if t not in vocab:
                vocab.append(t)

In [20]:
avocab = []
i = 0
for token in atoken_with_category:
    i += 1
    print(str((i/len(atoken_with_category))*100),end='\r')
    if str(token[1]+'-category') in top20_category_first:
        for t in token[0]:
            if t not in avocab:
                avocab.append(t)

In [21]:
len(vocab)

16026

In [ ]:
# topic_dictionary = corpora.Dictionary(topic_vocab)
# topic_corpus = [dictionary.doc2bow(t) for t in topic_vocab]
word2id = dict((v, i) for i, v in enumerate(vocab))
category2id = dict((c[:len(c)-9], i) for i, c in enumerate(top20_category_first))

In [21]:
aword2id = dict((v, i) for i, v in enumerate(avocab))
acategory2id = dict((c[:len(c)-9], i) for i, c in enumerate(top20_category_first))

In [23]:
i = 0
top20_category_token = []
for token in token_with_category:
    i += 1
    print(str((i/len(token_with_category))*100),end='\r')
    if str(token[1]+'-category') in top20_category_first:
        top20_category_token.append(token)

In [22]:
i = 0
atop20_category_token = []
for token in atoken_with_category:
    i += 1
    print(str((i/len(atoken_with_category))*100),end='\r')
    if str(token[1]+'-category') in top20_category_first:
        atop20_category_token.append(token)

86.22563549806881555520.16527440941345641.561124584568465.12171023084522

In [24]:
top20_category_token

[[['neural',
   'computing',
   'cancer',
   'drug',
   'development',
   'predicting',
   'mechanism',
   'action'],
  'chemoinformatics-databases'],
 [['assessment', 'protein', 'model', 'dimensional', 'profile'],
  'protein-structure-analysis'],
 [['membrane',
   'protein',
   'structure',
   'prediction',
   'hydrophobicity',
   'analysis',
   'positive',
   'inside',
   'rule'],
  'protein-sequence-analysis'],
 [['identify',
   'protein',
   'sequence',
   'fold',
   'known',
   'dimensional',
   'structure'],
  'protein-structure-analysis'],
 [['omim',
   'line',
   'mendelian',
   'inheritance',
   'man',
   'expert',
   'medical',
   'genetics'],
  'dna-sequence-databases'],
 [['efficient',
   'detection',
   'dimensional',
   'structural',
   'motif',
   'biological',
   'macromolecule',
   'vision',
   'technique'],
  'protein-structure-analysis'],
 [['visual', 'interface', 'program', 'linkage', 'analysis'],
  'population-genomics'],
 [['workbench', 'multiple', 'alignment', 'c

In [ ]:
X = [[0]*len(word2id)]*len(top20_category_token)
X = np.array(X)

In [23]:
aX = [[0]*len(aword2id)]*len(atop20_category_token)

In [24]:
aX = np.array(aX)

In [27]:
for i in range(len(top20_category_token)):
    for t in top20_category_token[i][0]:
        X[i][word2id[t]] +=1

In [27]:
for i in range(len(atop20_category_token)):
    for t in atop20_category_token[i][0]:
        aX[i][aword2id[t]] +=1

In [28]:
Y = []
for i in range(len(top20_category_token)):
    Y.append(category2id[top20_category_token[i][1]])

In [28]:
aY = []
for i in range(len(atop20_category_token)):
    aY.append(acategory2id[atop20_category_token[i][1]])

In [29]:
cutoff = int(len(top20_category_token)*(19/20))

print(cutoff,len(top20_category_token))

print('train on '+str(cutoff)+' docs, test on '+str(len(top20_category_token)-cutoff)+ ' docs')


14894 15678
train on 14894 docs, test on 784 docs


In [34]:
acutoff = int(len(atop20_category_token)*(4/5))

print(acutoff,len(atop20_category_token))

print('train on '+str(acutoff)+' docs, test on '+str(len(atop20_category_token)-acutoff)+ ' docs')


In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [12]:
clf = MultinomialNB()
clf.fit(X[:cutoff], Y[:cutoff])

NameError: name 'X' is not defined

In [31]:
print(clf.score(X[cutoff:],Y[cutoff:]))

0.5395408163265306


In [35]:
clf = MultinomialNB()
clf.fit(aX[:acutoff], aY[:acutoff])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [36]:
print(clf.score(aX[acutoff:],aY[acutoff:]))

0.5944479897894065
